In [81]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import xgboost as xgb
import optuna
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from IPython.display import display, HTML

# Configuration
pd.set_option('display.max_columns', None)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Check if data exists to avoid re-downloading
if 'pbp' not in locals():
    years = [2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
    print(f"📥 Downloading NFL Data for {years}...")
    try:
        schedule = nfl.import_schedules(years)
        pbp = nfl.import_pbp_data(years)
        print("✅ Data Loaded Successfully.")
    except Exception as e:
        print(f"❌ Error loading data: {e}")
else:
    print("✅ Data already loaded. Proceed to Cell 2.")

✅ Data already loaded. Proceed to Cell 2.


In [113]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# --- 1. FEATURE ENGINEERING ---
print("⚙️ Processing Stats (Restoring & Refining the Golden Era Model)...")

# Garbage Time Filter
pbp_clean = pbp[((pbp['pass'] == 1) | (pbp['rush'] == 1)) & (pbp['wp'] > 0.05) & (pbp['wp'] < 0.95)].dropna(subset=['epa', 'posteam', 'defteam', 'success', 'yards_gained'])

def get_team_stats(df, full_pbp):
    # 1. Base Stats
    gen = df.groupby(['season', 'week', 'posteam']).agg({'epa': 'mean', 'yards_gained': 'mean'}).reset_index().rename(columns={'posteam': 'team', 'epa': 'off_epa', 'yards_gained': 'off_ypp'})
    
    # 2. Early Down Success Rate (EDSR)
    edsr = df[df['down'].isin([1, 2])].groupby(['season', 'week', 'posteam'])['success'].mean().reset_index().rename(columns={'posteam': 'team', 'success': 'off_edsr'})

    # 3. Trench Warfare (Sack Rates)
    pass_plays = full_pbp[full_pbp['pass'] == 1]
    off_sacks = pass_plays.groupby(['season', 'week', 'posteam'])['sack'].mean().reset_index().rename(columns={'posteam': 'team', 'sack': 'off_sack_rate'})
    def_sacks = pass_plays.groupby(['season', 'week', 'defteam'])['sack'].mean().reset_index().rename(columns={'defteam': 'team', 'sack': 'def_sack_rate'})

    # 4. Special Teams, Turnovers & Penalties
    st = full_pbp[full_pbp['special_teams_play'] == 1].groupby(['season', 'week', 'posteam'])['epa'].mean().reset_index().rename(columns={'posteam': 'team', 'epa': 'st_epa'})
    
    tos = full_pbp.groupby(['season', 'week', 'posteam']).agg({'fumble_lost': 'sum', 'interception': 'sum'}).reset_index()
    tos['turnovers_lost'] = tos['fumble_lost'] + tos['interception']
    
    penalties = full_pbp[full_pbp['penalty'] == 1].groupby(['season', 'week', 'penalty_team']).agg({'penalty_yards': 'sum'}).reset_index().rename(columns={'penalty_team': 'team', 'penalty_yards': 'pen_yards'})
    
    # 5. Red Zone Efficiency
    rz = df[df['yardline_100'] <= 20].groupby(['season', 'week', 'posteam'])['epa'].mean().reset_index().rename(columns={'posteam': 'team', 'epa': 'off_rz_epa'})

    # Merge All
    merged = gen.merge(edsr, on=['season', 'week', 'team'], how='left')
    merged = merged.merge(off_sacks, on=['season', 'week', 'team'], how='left').merge(def_sacks, on=['season', 'week', 'team'], how='left')
    merged = merged.merge(st, on=['season', 'week', 'team'], how='left')
    merged = merged.merge(tos[['season', 'week', 'posteam', 'turnovers_lost']].rename(columns={'posteam': 'team'}), on=['season', 'week', 'team'], how='left')
    merged = merged.merge(penalties, on=['season', 'week', 'team'], how='left')
    merged = merged.merge(rz, on=['season', 'week', 'team'], how='left')
    
    # Pass Splits
    pass_df = df[df['pass'] == 1].groupby(['season', 'week', 'posteam'])['epa'].mean().reset_index().rename(columns={'posteam': 'team', 'epa': 'off_pass_epa'})
    merged = merged.merge(pass_df, on=['season', 'week', 'team'], how='left')
    
    return merged.fillna(0)

stats = get_team_stats(pbp_clean, pbp).sort_values(['team', 'season', 'week'])

# Rolling Averages
metrics = ['off_epa', 'off_ypp', 'off_pass_epa', 
           'off_edsr', 'off_sack_rate', 'def_sack_rate', 
           'st_epa', 'turnovers_lost', 'pen_yards', 'off_rz_epa']

for col in metrics:
    stats[f'{col}_long'] = stats.groupby(['team', 'season'])[col].transform(lambda x: x.shift(1).ewm(span=10).mean())
    stats[f'{col}_short'] = stats.groupby(['team', 'season'])[col].transform(lambda x: x.shift(1).ewm(span=3).mean())

# Pythagorean Wins & Luck
def add_standings(games_df):
    home = games_df[['season', 'week', 'home_team', 'home_score', 'away_score']].rename(columns={'home_team': 'team', 'home_score': 'pf', 'away_score': 'pa'})
    away = games_df[['season', 'week', 'away_team', 'away_score', 'home_score']].rename(columns={'away_team': 'team', 'away_score': 'pf', 'home_score': 'pa'})
    results = pd.concat([home, away]).sort_values(['team', 'season', 'week']).dropna()
    
    results['win'] = (results['pf'] > results['pa']).astype(int)
    results['cum_pf'] = results.groupby(['team', 'season'])['pf'].transform(lambda x: x.shift(1).cumsum())
    results['cum_pa'] = results.groupby(['team', 'season'])['pa'].transform(lambda x: x.shift(1).cumsum())
    results['cum_wins'] = results.groupby(['team', 'season'])['win'].transform(lambda x: x.shift(1).cumsum())
    results['games_played'] = results.groupby(['team', 'season'])['win'].transform(lambda x: x.shift(1).expanding().count()).fillna(0)
    
    results['pythag_wins'] = np.where(results['cum_pf'] == 0, 0, (results['cum_pf']**2.37) / ((results['cum_pf']**2.37) + (results['cum_pa']**2.37)))
    results['win_pct'] = np.where(results['games_played'] > 0, results['cum_wins'] / results['games_played'], 0)
    results['luck_metric'] = results['win_pct'] - results['pythag_wins']
    
    return results[['season', 'week', 'team', 'pythag_wins', 'luck_metric']].fillna(0)

standings = add_standings(schedule)
stats = stats.merge(standings, on=['season', 'week', 'team'], how='left').fillna(0)
stats = stats.dropna()

# QB Database
print("🏈 Building QB Database...")
qb_data = pbp_clean[pbp_clean['pass'] == 1].groupby(['season', 'posteam', 'name']).agg({'epa': 'mean', 'play_id': 'count'}).reset_index()
qb_data = qb_data[qb_data['play_id'] > 15]
qb_stability = qb_data.groupby(['season', 'posteam'])['epa'].std().reset_index().rename(columns={'epa': 'qb_volatility', 'posteam': 'team'}).fillna(0)
stats = stats.merge(qb_stability, on=['season', 'team'], how='left')

# Prepare Games
games = schedule[schedule['game_type'] == 'REG'].copy()
games = games.drop(columns=['home_rest', 'away_rest'], errors='ignore')
games['roof'] = games['roof'].map({'outdoors': 0, 'open': 0, 'closed': 1, 'dome': 1}).fillna(0)

# Expanding Window for Home Field Strength
games_sorted = games.sort_values(['season', 'week'])
home_results = games_sorted[['home_team', 'result']].rename(columns={'home_team': 'team'})
games['home_field_strength'] = home_results.groupby('team')['result'].transform(lambda x: x.shift(1).expanding().mean()).fillna(2.0)

# Rest Days
games['gameday'] = pd.to_datetime(games['gameday'])
rest_df = pd.concat([games[['season', 'week', 'gameday', 'home_team']].rename(columns={'home_team': 'team'}), 
                     games[['season', 'week', 'gameday', 'away_team']].rename(columns={'away_team': 'team'})]).sort_values(['team', 'gameday'])
rest_df['rest'] = (rest_df['gameday'] - rest_df.groupby('team')['gameday'].shift(1)).dt.days.fillna(7).clip(upper=14)
games = games.merge(rest_df[['season', 'week', 'team', 'rest']], left_on=['season', 'week', 'home_team'], right_on=['season', 'week', 'team']).rename(columns={'rest': 'home_rest'}).drop(columns=['team'])
games = games.merge(rest_df[['season', 'week', 'team', 'rest']], left_on=['season', 'week', 'away_team'], right_on=['season', 'week', 'team']).rename(columns={'rest': 'away_rest'}).drop(columns=['team'])

# Merge Stats
cols_base = [f'{m}{s}' for m in metrics for s in ['_long', '_short']] + ['qb_volatility', 'pythag_wins', 'luck_metric']
for side in ['home', 'away']:
    games = games.merge(stats[['season', 'week', 'team'] + cols_base], left_on=['season', 'week', f'{side}_team'], right_on=['season', 'week', 'team'])
    games.rename(columns={c: f'{side}_{c}' for c in cols_base}, inplace=True)
    games.drop(columns=['team'], inplace=True)

# --- FEATURE ENGINEERING (Golden Era - Trimmed) ---
games['qb_diff'] = games['home_off_pass_epa_long'] - games['away_off_pass_epa_long']
games['edsr_diff'] = games['home_off_edsr_long'] - games['away_off_edsr_long']
games['ypp_diff'] = games['home_off_ypp_long'] - games['away_off_ypp_long']
games['pythag_diff'] = games['home_pythag_wins'] - games['away_pythag_wins']
games['rest_diff'] = games['home_rest'] - games['away_rest']
games['st_diff'] = games['home_st_epa_long'] - games['away_st_epa_long']
games['turnover_diff'] = games['home_turnovers_lost_long'] - games['away_turnovers_lost_long']
games['rz_diff'] = games['home_off_rz_epa_long'] - games['away_off_rz_epa_long']
games['penalty_diff'] = games['home_pen_yards_long'] - games['away_pen_yards_long']

# Trench Warfare
games['sack_mismatch_home'] = games['home_off_sack_rate_long'] - games['away_def_sack_rate_long']
games['sack_mismatch_away'] = games['away_off_sack_rate_long'] - games['home_def_sack_rate_long']

# --- 2. WALK-FORWARD VALIDATION ---
# Removed: div_game, momentum_diff, luck_diff (Noise reduction)
X_cols = [
    'qb_diff', 'edsr_diff', 'ypp_diff', 'pythag_diff', 'rest_diff',
    'sack_mismatch_home', 'sack_mismatch_away',
    'st_diff', 'turnover_diff', 'rz_diff', 'penalty_diff',
    'home_field_strength', 'roof',
    'home_qb_volatility', 'away_qb_volatility'
]
target = 'result'

# Monotonic Constraints
mono_constraints = (
    1,  # qb_diff
    1,  # edsr_diff
    1,  # ypp_diff
    1,  # pythag_diff
    1,  # rest_diff
    -1, # sack_mismatch_home
    1,  # sack_mismatch_away
    1,  # st_diff
    -1, # turnover_diff
    1,  # rz_diff
    -1, # penalty_diff
    1,  # home_field
    0,  # roof
    -1, # home_qb_volatility
    1   # away_qb_volatility
)

print(f"🚀 Starting Walk-Forward Validation on {len(X_cols)} Refined Features...")

validation_results = []
models = {}
test_seasons = [2023, 2024] 

for test_year in test_seasons:
    train_data = games[(games['season'] < test_year) & (games['season'] >= 2018)].dropna(subset=X_cols + [target])
    test_data = games[games['season'] == test_year].dropna(subset=X_cols + [target])
    
    X_train, y_train = train_data[X_cols], train_data[target]
    X_test, y_test = test_data[X_cols], test_data[target]
    
    # Target Clip: ±21
    y_train = y_train.clip(-21, 21)
    
    model = xgb.XGBRegressor(
        n_estimators=2000, 
        learning_rate=0.01, 
        max_depth=3,              
        min_child_weight=20,      
        reg_alpha=0.5,            
        subsample=0.5,            
        colsample_bytree=0.5,     
        monotone_constraints=mono_constraints,
        early_stopping_rounds=50, 
        n_jobs=-1, 
        objective='reg:squarederror'
    )
    
    split = int(len(X_train) * 0.9)
    X_tr, y_tr = X_train.iloc[:split], y_train.iloc[:split]
    X_val, y_val = X_train.iloc[split:], y_train.iloc[split:]
    
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    
    val_preds = model.predict(X_val)
    bias = np.mean(val_preds - y_val)
    preds = model.predict(X_test) - bias 
    
    test_data['pred'] = preds
    test_data['error'] = test_data['pred'] - test_data['result']
    test_data['abs_error'] = test_data['error'].abs()
    validation_results.append(test_data)
    models[test_year] = model

full_audit = pd.concat(validation_results)
rmse = np.sqrt(mean_squared_error(full_audit['result'], full_audit['pred']))
mae = full_audit['abs_error'].mean()

# --- 3. DIAGNOSTICS ---
def ascii_bar(val, max_val, width=15): return "█" * int((val / max_val) * width) if max_val > 0 else ""

print("\n" + "="*60 + "\n🕵️  WALK-FORWARD TEST RESULTS (2023-2024)\n" + "="*60)
print(f"1. PERFORMANCE METRICS")
print(f"   RMSE: {rmse:.4f}")
print(f"   MAE:  {mae:.4f}")

print(f"\n2. BETTING SIMULATION (Refined Golden Model)")
full_audit['spread_line'] = full_audit['spread_line'].fillna(0)
full_audit['home_cover'] = np.where(full_audit['result'] > full_audit['spread_line'], 1, 0)
full_audit['away_cover'] = np.where(full_audit['result'] < full_audit['spread_line'], 1, 0)
full_audit['push'] = np.where(full_audit['result'] == full_audit['spread_line'], 1, 0)

for threshold in [1.5, 2.5, 3.5]:
    bets = full_audit.copy()
    bets['bet_home'] = np.where(bets['pred'] > (bets['spread_line'] + threshold), 1, 0)
    bets['bet_away'] = np.where(bets['pred'] < (bets['spread_line'] - threshold), 1, 0)
    
    wins = len(bets[(bets['bet_home'] == 1) & (bets['home_cover'] == 1)]) + len(bets[(bets['bet_away'] == 1) & (bets['away_cover'] == 1)])
    losses = len(bets[(bets['bet_home'] == 1) & (bets['away_cover'] == 1)]) + len(bets[(bets['bet_away'] == 1) & (bets['home_cover'] == 1)])
    pushes = len(bets[(bets['bet_home'] == 1) & (bets['push'] == 1)]) + len(bets[(bets['bet_away'] == 1) & (bets['push'] == 1)])
    
    win_pct = wins / (wins + losses) if (wins + losses) > 0 else 0
    print(f"   Edge > {threshold}: {wins}-{losses}-{pushes} ({win_pct:.2%})")

print(f"\n3. FEATURE IMPORTANCE")
latest_model = models[2024]
imps = pd.DataFrame({'f': X_cols, 'i': latest_model.feature_importances_}).sort_values('i', ascending=False)
for _, r in imps.iterrows(): print(f"{r['f']:<25} | {r['i']:.4f} | {ascii_bar(r['i'], imps['i'].max())}")

# Final Train
print("\n🚀 Retraining Master Model on ALL Data (2018-2024)...")
y_final = games[target].clip(-21, 21)
final_model = xgb.XGBRegressor(n_estimators=2000, learning_rate=0.01, max_depth=3, min_child_weight=20, reg_alpha=0.5, subsample=0.5, colsample_bytree=0.5, monotone_constraints=mono_constraints, n_jobs=-1, objective='reg:squarederror')
final_model.fit(games[X_cols], y_final, verbose=False)
print("✅ Master Model Ready.")

⚙️ Processing Stats (Restoring & Refining the Golden Era Model)...
🏈 Building QB Database...
🚀 Starting Walk-Forward Validation on 15 Refined Features...

🕵️  WALK-FORWARD TEST RESULTS (2023-2024)
1. PERFORMANCE METRICS
   RMSE: 13.5290
   MAE:  10.4324

2. BETTING SIMULATION (Refined Golden Model)
   Edge > 1.5: 187-192-12 (49.34%)
   Edge > 2.5: 146-144-8 (50.34%)
   Edge > 3.5: 113-95-5 (54.33%)

3. FEATURE IMPORTANCE
pythag_diff               | 0.1350 | ███████████████
edsr_diff                 | 0.1300 | ██████████████
qb_diff                   | 0.1036 | ███████████
ypp_diff                  | 0.0976 | ██████████
home_field_strength       | 0.0913 | ██████████
home_qb_volatility        | 0.0592 | ██████
away_qb_volatility        | 0.0545 | ██████
st_diff                   | 0.0527 | █████
sack_mismatch_home        | 0.0516 | █████
rz_diff                   | 0.0445 | ████
sack_mismatch_away        | 0.0441 | ████
turnover_diff             | 0.0408 | ████
rest_diff                

In [114]:
# --- CONFIGURATION ---
FIX_VEGAS_SIGNS = True 
QB_OVERRIDES = {} 

def run_dashboard(model, schedule_df, pbp_stats_df, qb_db, feature_cols):
    current_season = 2025
    upcoming = schedule_df[(schedule_df['season'] == current_season) & (schedule_df['result'].isna())]
    if upcoming.empty: return print("⚠️ No upcoming games found.")

    next_week = upcoming['week'].min()
    week_df = upcoming[upcoming['week'] == next_week].copy()
    latest_stats = pbp_stats_df.sort_values(['season', 'week']).groupby('team').tail(1).set_index('team')
    
    recent_pbp = pbp[(pbp['season'] == current_season) & (pbp['week'] >= next_week - 3)]
    default_starters = recent_pbp[recent_pbp['pass']==1].groupby(['posteam', 'name'])['play_id'].count().reset_index().sort_values('play_id', ascending=False).groupby('posteam').head(1)
    default_starters = default_starters.set_index('posteam')['name'].to_dict()
    
    full_schedule = schedule_df.copy()
    full_schedule['gameday'] = pd.to_datetime(full_schedule['gameday'])
    def get_rest(team, target_week):
        tg = full_schedule[((full_schedule['home_team'] == team) | (full_schedule['away_team'] == team)) & (full_schedule['season'] == current_season)].sort_values('gameday')
        pg = tg[tg['week'] < target_week]
        if pg.empty: return 7
        return min((tg[tg['week'] == target_week].iloc[0]['gameday'] - pg.iloc[-1]['gameday']).days, 14)

    def get_qb_epa(team, qb_name):
        try:
            row = qb_db[(qb_db['season'] == current_season) & (qb_db['posteam'] == team) & (qb_db['name'] == qb_name)]
            if not row.empty: return row.iloc[0]['epa']
            row = qb_db[(qb_db['season'] == current_season - 1) & (qb_db['name'] == qb_name)]
            if not row.empty: return row.iloc[0]['epa']
            return None 
        except: return None

    betting_data = []
    for _, game in week_df.iterrows():
        home, away = game['home_team'], game['away_team']
        raw_spread = game['spread_line']
        vegas_line = 0.0 if pd.isna(raw_spread) else (-1 * raw_spread if FIX_VEGAS_SIGNS else raw_spread)
        vegas_display = "N/A" if pd.isna(raw_spread) else f"{home} {vegas_line:.1f}" if vegas_line < 0 else f"{home} +{vegas_line:.1f}"

        if home not in latest_stats.index or away not in latest_stats.index: continue
        h, a = latest_stats.loc[home], latest_stats.loc[away]

        h_qb_name = QB_OVERRIDES.get(home, default_starters.get(home, "Unknown"))
        a_qb_name = QB_OVERRIDES.get(away, default_starters.get(away, "Unknown"))
        h_qb_epa = get_qb_epa(home, h_qb_name)
        a_qb_epa = get_qb_epa(away, a_qb_name)
        
        h_pass_epa = h_qb_epa if h_qb_epa is not None else h.get('off_pass_epa_long', 0)
        a_pass_epa = a_qb_epa if a_qb_epa is not None else a.get('off_pass_epa_long', 0)
        
        data = {}
        data['qb_diff'] = h_pass_epa - a_pass_epa
        data['edsr_diff'] = h.get('off_edsr_long', 0) - a.get('off_edsr_long', 0)
        data['ypp_diff'] = h.get('off_ypp_long', 0) - a.get('off_ypp_long', 0)
        data['pythag_diff'] = h.get('pythag_wins', 0.5) - a.get('pythag_wins', 0.5)
        data['rest_diff'] = get_rest(home, next_week) - get_rest(away, next_week)
        data['st_diff'] = h.get('st_epa_long', 0) - a.get('st_epa_long', 0)
        data['turnover_diff'] = h.get('turnovers_lost_long', 0) - a.get('turnovers_lost_long', 0)
        data['rz_diff'] = h.get('off_rz_epa_long', 0) - a.get('off_rz_epa_long', 0)
        data['penalty_diff'] = h.get('pen_yards_long', 0) - a.get('pen_yards_long', 0)
        data['sack_mismatch_home'] = h.get('off_sack_rate_long', 0) - a.get('def_sack_rate_long', 0)
        data['sack_mismatch_away'] = a.get('off_sack_rate_long', 0) - h.get('def_sack_rate_long', 0)
        data['div_game'] = 1 if game['div_game'] == 1 else 0
        data['home_field_strength'] = 2.0 
        r_val = {'outdoors': 0, 'open': 0, 'closed': 1, 'dome': 1}.get(game['roof'], 0)
        data['roof'] = r_val
        data['home_qb_volatility'] = h.get('qb_volatility', 0)
        data['away_qb_volatility'] = a.get('qb_volatility', 0)
        
        input_df = pd.DataFrame([data]).reindex(columns=feature_cols, fill_value=0)
        raw_pred = -1 * final_model.predict(input_df)[0]
        
        fair_line = raw_pred
        if fair_line > 21: fair_line = 21
        if fair_line < -21: fair_line = -21
        
        if vegas_display != "N/A":
            diff = fair_line - vegas_line
            if abs(diff) > 10: fair_line = vegas_line + (diff * 0.5) 
        
        edge = vegas_line - fair_line if vegas_display != "N/A" else 0.0
        action = "PASS"
        confidence = ""
        
        if vegas_display != "N/A":
            if abs(edge) >= 3.5:
                confidence = "🔥 STRONG"
                action = f"BET {home}" if edge > 0 else f"BET {away}"
            elif abs(edge) >= 2.0:
                confidence = "⚠️ LEAN"
                action = f"BET {home}" if edge > 0 else f"BET {away}"
        
        f_str = f"{home} {fair_line:.1f}" if fair_line < 0 else f"{home} +{fair_line:.1f}"
        final_note = ""
        if h_qb_name == "Unknown" or a_qb_name == "Unknown": final_note += "⚠️ Unknown QB "
        if h_qb_epa is None: final_note += f"⚠️ Using Team Stats for {home} "
        
        betting_data.append({'Matchup': f"{away} @ {home}", 'QBs': f"{a_qb_name} vs {h_qb_name}", 'Vegas': vegas_display, 'Fair_Line': f_str, 'Edge': round(edge, 1) if vegas_display != "N/A" else "N/A", 'Action': action, 'Conf': confidence, 'Note': final_note})
        
    df = pd.DataFrame(betting_data)
    print(f"\n🏈 WEEK {next_week} HANDICAPPER SHEET")
    def style_action(val): return 'color: #D00000; font-weight: bold;' if "BET" in val else 'color: black;'
    def style_conf(val): return 'background-color: #ffcccc; font-weight: bold;' if "STRONG" in val else ''
    
    df['Sort'] = df['Edge'].apply(lambda x: abs(x) if isinstance(x, (int, float)) else 0)
    styled = df.sort_values('Sort', ascending=False).drop(columns=['Sort']).style.applymap(style_action, subset=['Action']).applymap(style_conf, subset=['Conf']).hide(axis='index')
    display(styled)

run_dashboard(final_model, schedule, stats, qb_db, X_cols)


🏈 WEEK 15 HANDICAPPER SHEET


Matchup,QBs,Vegas,Fair_Line,Edge,Action,Conf,Note
NYJ @ JAX,T.Taylor vs T.Lawrence,JAX -14.0,JAX -4.3,-9.700000,BET NYJ,🔥 STRONG,
CLE @ CHI,S.Sanders vs C.Williams,CHI -7.5,CHI -16.7,9.200000,BET CHI,🔥 STRONG,
ARI @ HOU,J.Brissett vs C.Stroud,HOU -10.5,HOU -1.6,-8.900000,BET ARI,🔥 STRONG,
TEN @ SF,C.Ward vs B.Purdy,SF -12.5,SF -21.0,8.500000,BET SF,🔥 STRONG,
MIN @ DAL,J.McCarthy vs D.Prescott,DAL -5.5,DAL -13.2,7.800000,BET DAL,🔥 STRONG,
BUF @ NE,J.Allen vs D.Maye,NE +1.5,NE -5.8,7.300000,BET NE,🔥 STRONG,
IND @ SEA,D.Jones vs S.Darnold,SEA -14.0,SEA -8.5,-5.500000,BET IND,🔥 STRONG,
BAL @ CIN,L.Jackson vs J.Burrow,CIN +2.5,CIN -2.5,5.000000,BET CIN,🔥 STRONG,
LV @ PHI,G.Smith vs J.Hurts,PHI -12.5,PHI -9.1,-3.400000,BET LV,⚠️ LEAN,
MIA @ PIT,T.Tagovailoa vs A.Rodgers,PIT -3.0,PIT -0.2,-2.800000,BET MIA,⚠️ LEAN,


In [118]:
import pandas as pd
import pickle
import os
import numpy as np
from tabulate import tabulate

CACHE_PATH = "data/nfl_db.pkl"

print("🔍 DIAGNOSTIC CHECK...")

if not os.path.exists(CACHE_PATH):
    print("❌ No DB found. Please run update_db.py first.")
else:
    with open(CACHE_PATH, 'rb') as f:
        db = pickle.load(f)
    
    games = db['games_df']
    current_season = db['current_season']
    print(f"✅ Loaded DB. Season: {current_season}")
    
    # 1. Check Game Count
    season_games = games[games['season'] == current_season]
    print(f"📊 Total Games in {current_season}: {len(season_games)}")
    
    # 2. Check Results & Spreads
    graded = season_games[season_games['result'].notna()]
    with_odds = season_games[season_games['spread_line'].notna()]
    print(f"   - Completed Games: {len(graded)}")
    print(f"   - Games with Odds: {len(with_odds)}")
    
    # 3. Check Features (The likely culprit)
    # Check if 'qb_diff' is all zeros
    non_zero_feats = season_games[season_games['qb_diff'] != 0]
    print(f"   - Games with valid QB Stats: {len(non_zero_feats)}")
    
    if len(non_zero_feats) == 0:
        print("\n⚠️ CRITICAL ISSUE FOUND: All Feature Stats are ZERO.")
        print("   This means the Merge between Schedule and Stats failed.")
        print("   Likely cause: Team Name Mismatch (e.g. 'WAS' vs 'WSH').")
    
    # 4. PREVIEW DATA
    print("\n🧐 Sample Data Row (First 5 columns):")
    print(season_games[['week', 'home_team', 'away_team', 'spread_line', 'qb_diff']].tail(3))
    
    # --- AUTOMATIC REPAIR ATTEMPT ---
    if len(non_zero_feats) == 0:
        print("\n🛠️  ATTEMPTING REPAIR...")
        # Reload raw data
        import nfl_data_py as nfl
        
        # Mapping Dictionary for common mismatches
        team_map = {'WSH': 'WAS', 'HST': 'HOU', 'BLT': 'BAL', 'CLV': 'CLE', 'ARZ': 'ARI'}
        
        # 1. Re-run stats engine manually here to test
        print("   -> Fetching fresh PBP...")
        pbp = nfl.import_pbp_data([current_season])
        
        # FIX TEAM NAMES IN PBP
        pbp['posteam'] = pbp['posteam'].replace(team_map)
        pbp['defteam'] = pbp['defteam'].replace(team_map)
        
        print("   -> Recalculating Stats...")
        # (Simplified Stats Logic for Repair)
        stats = pbp.groupby(['season', 'week', 'posteam']).agg({'epa': 'mean'}).reset_index().rename(columns={'posteam': 'team', 'epa': 'off_epa'})
        stats['off_epa_long'] = stats.groupby('team')['off_epa'].transform(lambda x: x.shift(1).ewm(span=10).mean())
        
        # Merge back to games
        games_repair = games.copy()
        # Drop old broken columns if they exist
        if 'home_off_epa_long' in games_repair.columns:
            games_repair = games_repair.drop(columns=['home_off_epa_long', 'away_off_epa_long'])
            
        games_repair = games_repair.merge(stats[['season', 'week', 'team', 'off_epa_long']], left_on=['season', 'week', 'home_team'], right_on=['season', 'week', 'team'], how='left').rename(columns={'off_epa_long': 'home_off_epa_long'}).drop(columns=['team'])
        games_repair = games_repair.merge(stats[['season', 'week', 'team', 'off_epa_long']], left_on=['season', 'week', 'away_team'], right_on=['season', 'week', 'team'], how='left').rename(columns={'off_epa_long': 'away_off_epa_long'}).drop(columns=['team'])
        
        # Check if we fixed it
        fixed_count = len(games_repair[games_repair['home_off_epa_long'].notna()])
        print(f"   -> Repair Result: {fixed_count} games now have EPA stats.")
        
        if fixed_count > 0:
            print("✅ Repair Successful. Please update 'update_db.py' to handle team name mapping.")

    # 5. FORCE PREDICTION CHECK
    print("\n🎲 RUNNING TEST PREDICTION...")
    model = db['model']
    # Pick a game with odds
    if not with_odds.empty:
        test_game = with_odds.iloc[-1]
        X = pd.DataFrame([test_game[db['model'].feature_names_in_]])
        pred = -1 * model.predict(X)[0]
        print(f"   Matchup: {test_game['away_team']} @ {test_game['home_team']}")
        print(f"   Vegas: {test_game['spread_line']}")
        print(f"   Model: {pred:.2f}")
        print(f"   Edge:  {(-1 * test_game['spread_line']) - pred:.2f}")
    else:
        print("   No games with odds found to test.")

✅ update_db.py FIXED! Run 'python update_db.py' now.
